<a href="https://colab.research.google.com/github/vithal-4u/ML-Experiments/blob/master/smsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Reading CSV file
message_data = pd.read_csv("smsDataFormated.csv",names=['Label','Body'],skiprows=1)
message_data.head()

message_data = message_data.rename(columns = {'Index':'Label','Body':'message'})
message_data.groupby('Label').describe()

message_data

message_label = message_data['Label'].copy()
message_body = message_data['message']
print("Label ---", message_label)
print("Body ---", message_body)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Label --- 0       Network
1       Network
2       Network
3       Network
4       Network
         ...   
4626      Trans
4627      Trans
4628      Trans
4629      Trans
4630      Trans
Name: Label, Length: 4631, dtype: object
Body --- 0        You have 1 missed call(s) from +17342999521 ....
1                                                 "300457
2                        Your single-use eBay PIN is 9013
3        You have 1 missed call(s) from +911166793800 ...
4        You have 1 missed call(s) from +911166795200 ...
                              ...                        
4626      "Your a/c no. XXXXXXXX2251 is debited for Rs. 3
4627      "Your a/c no. XXXXXXXX2251 is debited for Rs. 6
4628                                       "Dear Customer
4629     "Personal Loan HDFC Bank offers For CTS FLAT1...
4630     "PNR-4634235664 Trn:12673 Dt:11-08-2016 Frm M...
Name: message, 

In [0]:
def text_preprocess(text):
    text = str(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [7]:
message_body = message_body.apply(text_preprocess)
message_body

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(message_body)
message_mat

<4631x6969 sparse matrix of type '<class 'numpy.float64'>'
	with 46798 stored elements in Compressed Sparse Row format>

In [0]:
#Splitting the data into test and train
from sklearn.model_selection import train_test_split
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_label, 
                                                        test_size=0.2, random_state=0)

In [9]:
#Creating LogisticRegression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
# Test the model with own data
pred = Spam_model.predict(message_test)
print("accuracy_score -- ",accuracy_score(spam_nospam_test,pred))
few_test = vectorizer.transform( ["+918870782800 is now available to take calls. To CALL BACK dial +918870782800.For Top HelloTunes, Airtel Call Manager or Live aarti,call 567892 (tollfree)", 
                                "Akshara and 6 others have added their photos & also match your partner preference. View photos on EzhavaMatrimony by visiting www.cmatri.in/?kNw8dB~n~grNC",
                                "Rs.1000.00 was withdrawn using your HDFC Bank Card ending 0449 on 2017-04-01:21:31:34 at +CTS THORAIPAKKAM OATM. Avl bal: Rs.18834.44",
                                "Trying to loose weight? Meet our Bariartic Surgeon ¡ Apollo Spectra Hospital MRC Nagar.Spectra Obesity Check ¡ Rs 840.T & C apply. Call 66862000.",
                                "Dear Associate, Cognizant, offices in Chennai and Coimbatore will remain closed today, Dec 6, 2016. Note: Associates working on CIS, BPS and 24/7 support projects, please contact your manager for instructions."] )

pred_few = Spam_model.predict(few_test)
print(pred_few)

accuracy_score --  0.9050701186623517
['Network' 'Promotional' 'Trans' 'Promotional' 'Promotional']
